<a href="https://colab.research.google.com/github/garanews/garayara/blob/main/create_data_widget.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# INSTALLAZIONE LIBRERIE

In [1]:
!pip install requests requests-toolbelt

     |████████████████████████████████| 54 kB 2.3 MB/s 


## RECUPERO TOKEN CON LOGIN

In [2]:
import requests
import json
from requests_toolbelt.multipart.encoder import MultipartEncoder
from pathlib import Path
from datetime import datetime
import getpass

In [17]:
user = input("user: ")
password = getpass.getpass("password: ")

eve_url = "https://cis-backend.westeurope.cloudapp.azure.com"
api_url = "https://cis-backend.westeurope.cloudapp.azure.com/api"
token = None

alert_id = None
tweet_id = None
file_id = None

session = requests.session()
req = session.post(f"{api_url}/auth/login", json={"username": user, "password": password})

if req.status_code == 200:
    token = req.json()["data"]["access_token"]
    session.headers['Authorization'] = f"Bearer {token}"
    print("Token ok")

user: test
password: ··········
Token ok


## CREA ALERT

In [6]:
def create_alert():
  alert = {
      "entity_type": "alert",
      "type": "send_toast_message",
      "username": "test",
      "from": "twitter",
      "status": "new",
      "severity": "high",
      "labels": ["tweet"],
      "title": "New alert from twitter",
      "message": "New UAC-0056 activity: There’s a Go Elephant in the room, new hash: 1ce85d7be2e0717b79fbe0132e6851d81d0478dba563991b3404be9e58d745b1",
      "rule_id": "",
      "x-sources": [{
          "source_name" : "default",
          "classification" : 0,
          "releasability" : 0,
          "tlp" : 0
      }],
  }
  session.headers['Content-Type'] = 'application/json'
  req = session.post(f"{eve_url}/alerts", json=alert)
  if req.ok:
      alert_id = req.json()["_id"]
      print("Alert ID:", alert_id)
      return alert_id
  elif req.status_code == 409:
      alert_id = req.json()["_error"]["message"]["_id"]
      print("Alert ID:", alert_id)
      return alert_id
  print(req.json())
  return None
      

## CREA TWEET

In [7]:
def create_tweet():
  tweet = {
      "author_name": "CYBER OSINT",
      "author_username": "@Cyber_O51NT",
      "preview": "New UAC-0056 activity: There’s a Go Elephant in the room, new hash: 1ce85d7be2e0717b79fbe0132e6851d81d0478dba563991b3404be9e58d745b1",
      "description": "New UAC-0056 activity: There’s a Go Elephant in the room, new hash: 1ce85d7be2e0717b79fbe0132e6851d81d0478dba563991b3404be9e58d745b1",
      "x-sources": [{
          "source_name" : "default",
          "classification" : 0,
          "releasability" : 0,
          "tlp" : 0
      }],
  }

  session.headers['Content-Type'] = 'application/json'
  req = session.post(f"{eve_url}/x-tweets", json=tweet)
  if req.ok:
      tweet_id = req.json()["_id"]
      print("Tweets ID:", tweet_id)
      return tweet_id
  elif req.status_code == 409:
      tweet_id = req.json()["_error"]["message"]["_id"]
      print("Tweets ID:", tweet_id)
      return tweet_id
  print(req.json())
  return None

# CREA FILE

In [8]:
def create_file():
  filename = "tweet.csv"
  with open(filename, "w") as f:
    f.write("New UAC-0056 activity: There’s a Go Elephant in the room, new hash: 1ce85d7be2e0717b79fbe0132e6851d81d0478dba563991b3404be9e58d745b1")

  payload = {
    "content": (Path(filename).name, open(filename, 'rb'), 'application/csv'),
    "filename": Path(filename).name,
    "entity_type": "x_file",
    "x-source.source_name": "default",
    "x-source.classification": "0",
    "x-source.releasability": "0",
    "x-source.tlp": "0"
  }
  mp_encoder = MultipartEncoder(fields=payload, encoding='utf-8')
  session.headers['Content-Type'] = mp_encoder.content_type
  req = requests.post(f"{eve_url}/x_files", data=mp_encoder, headers=session.headers)

  if req.ok:
      file_id = req.json()["_id"]
      print("File ID:", file_id)
      return file_id
  elif req.status_code == 409:
      file_id = req.json()["_error"]["message"]["_id"]
      print("File ID:", file_id)
      return file_id
  print(req.json()["_error"]["message"])
  return None

# CREA RELAZIONE TRA TWEET E FILE (CONTENUTO RAW DEL TWEET)

In [9]:
def create_tweet_file_relationship(tweet_id, file_id):
  payload = {
    "confidence": 100,
    "relationship_type": "related-to",
    "source_ref": tweet_id,
    "source_type": "x-tweets",
    "spec_version": "2.1",
    "target_ref": file_id,
    "target_type": "x_files",
    "type": "relationship"
  }

  session.headers['Content-Type'] = 'application/json'
  req = session.post(f"{eve_url}/relationships", json=payload)
  if req.ok or req.status_code == 409:
      print("Relationship ok")
  else:
      print(req.json()["_error"])

# CREA RELAZIONE TRA ALERT E TWEET

In [10]:
def create_alert_tweet_relationship(alert_id, tweet_id):
  payload = {
    "confidence": 100,
    "relationship_type": "related-to",
    "source_ref": alert_id,
    "source_type": "alerts",
    "spec_version": "2.1",
    "target_ref": tweet_id,
    "target_type": "x-tweets",
    "type": "relationship"
  }

  session.headers['Content-Type'] = 'application/json'
  req = session.post(f"{eve_url}/relationships", json=payload)
  if req.ok or req.status_code == 409:
      print("Relationship ok")
  else:
      print(req.json()["_error"])

# CLEAN

In [11]:
def clean_vt_data():
  data= {
      "217.182.197.11": ("ipv4-addresses", "value"),
      "40.107.21.53": ("ipv4-addresses", "value"), 
      "mdfi.gov.ua": ("domain-names", "value"),
      "1ce85d7be2e0717b79fbe0132e6851d81d0478dba563991b3404be9e58d745b1": ("x-files-metadata", "sha256")
  }

  for (k, v) in data.items():
      payload = {
          "where": json.dumps({v[1]: k}),
          "projection": json.dumps({"_etag": 1, "_id": 1, v[1]: 1})
      }
      print(f"\tsearching: {k}")
      req = session.get(f"{eve_url}/{v[0]}", params=payload)
      if req.ok and len(req.json()["_items"])>0:
          data = req.json()["_items"][0]
          session.headers['If-Match'] = data["_etag"]
          print(f"{eve_url}/{v[0]}/{data['_id']}")
          req = session.delete(f"{eve_url}/{v[0]}/{data['_id']}")
          if req.ok:
              print(f'\t\t Item {data[v[1]]} with id {data["_id"]} deleted')
          else:
              print(f"\t\t error deleting item: {req.text}")
          
      else:
          print("\t\t not found")

In [12]:
def del_item(k, v):
      req = session.get(f"{eve_url}/{v}/{k}")
      print(f"\tsearching: {k}")
      if req.ok:
          data = req.json()
          session.headers['If-Match'] = data["_etag"]
          req = session.delete(f"{eve_url}/{v}/{data['_id']}")
          if req.ok:
              print(f'\t\tItem with id {data["_id"]} deleted')
          else:
              print(f"\t\t error deleting item: {req.text}")
      else:
          print("\t\tnot found")
def clean_data(alert_id, tweet_id, file_id):
    data= {
        alert_id: "alerts",
        tweet_id: "x-tweets", 
        file_id: "x_files"
    }
    print(data)
    for (k, v) in data.items():
        del_item(k, v)
    session.headers['If-Match'] = None

In [22]:
def clean_data_relationships(alert_id, tweet_id, file_id):
  data= {
    alert_id: "alerts",
    tweet_id: "x-tweets", 
    file_id: "x_files"
  }

  print(data)
  for (k, v) in data.items():
      payload = {
        "where": json.dumps({"$or": [{"source_ref": k, "source_type": v}, {"target_ref": k, "target_type": v}]})
      }
      req_one = session.get(f"{eve_url}/relationships", params=payload)
      if req_one.ok:
        relationships = req_one.json()["_items"]
        for data in relationships:
          tac_id = None
          if "alerts" in data["target_type"] and "threat-analysis-case" in data["source_type"]:
            tac_id = data["source_ref"]
          if "alerts" in data["source_type"] and "threat-analysis-case" in data["target_type"]:
            tac_id = data["target_ref"]
          if tac_id:
            del_item(tac_id, "threat-analysis-case")
          del_item(data['_id'], "relationships")
      else:
        print(req_one.json())

In [18]:
import ipywidgets as widgets
from IPython.display import display
button_create = widgets.Button(description="CREATE DATA!", button_style='success')
button_delete = widgets.Button(description="DELETE DATA!", button_style='danger')
output_create = widgets.Output()
output_delete = widgets.Output()

def create(b):
  global alert_id
  global tweet_id
  global file_id
  with output_create:
    output_create.clear_output()
    alert_id = create_alert()
    tweet_id = create_tweet()
    file_id = create_file()
    create_tweet_file_relationship(tweet_id, file_id)
    create_alert_tweet_relationship(alert_id, tweet_id)
    print("DONE")

def delete(b):
  global alert_id
  global tweet_id
  global file_id
  with output_delete:
    output_delete.clear_output()
    clean_vt_data()
    clean_data_relationships(alert_id, tweet_id, file_id)
    clean_data(alert_id, tweet_id, file_id)
    print("DONE")

button_create.on_click(create)
button_delete.on_click(delete)
display(button_create, output_create)
display(button_delete, output_delete)

Button(button_style='success', description='CREATE DATA!', style=ButtonStyle())

Output()

Button(button_style='danger', description='DELETE DATA!', style=ButtonStyle())

Output()